In [61]:
import pandas as pd
import math
import networkx
import os
from parsel import Selector

In [21]:
df = pd.read_csv("../spider/data.csv",encoding='utf-8',index_col="title")
path = "../spider/htmls"
files = os.listdir(path)

In [50]:
# 存储一个网页链接到一个列表的映射，列表中的元素为链接对应HTML文档中可以跳转到的链接
url_dict = {}

In [53]:
for file in files:
    title = str(file).split(".html")[0]
    url = df.loc[title].url
    with open(os.path.join(path,file),'r',encoding='utf-8-sig') as f :
        text = f.read()
        selector = Selector(text)
        url_list = []
        url_list.extend(selector.css('a::attr(href)').getall())
        url_dict[url] = url_list

In [62]:
# 计算pageRank：
def get_page_rank():
    digraph = networkx.DiGraph()
    for url, url_list in url_dict.items():
        for _url in url_list:
            if _url in df.url.values:
                digraph.add_edge(url, _url)
    result = networkx.pagerank(digraph, alpha=0.85)
    page_rank_df = pd.Series(result, name='page_rank')
    page_rank_df = page_rank_df.apply(lambda x: math.log(x * 10000, 10) + 1)  # 将page_rank列所有数值*10000后，取10的对数，再+1保证数值大于1（用于缩小PageRank极差，避免权重过大）
    page_rank_df.index.name = 'url'
    page_rank_df.to_csv("./page_rank.csv", encoding='utf-8-sig')

In [63]:
get_page_rank()